In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import random
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image
from torchvision import models
from sklearn.utils import shuffle
import sklearn
from os import listdir
import cv2
import time
import copy
from skimage import exposure
from skimage.exposure import match_histograms
from google.colab.patches import cv2_imshow
from torch.autograd import Variable
from random import sample
from torch import nn
from torch import autograd
import torch.nn.functional as F

In [ ]:
# 讀入所有原始資料
def load_all_img(AorB, class_num, p_train_num, p_test_num):
  path = "/content/drive/MyDrive/LiteonRacingData/" + str(AorB) + "/" + str(class_num) + "/" 
  img_train = []
  img_test = []
  f_name = []
  #ori_imgName = []
  for f in listdir(path):
    if f == "desktop.ini":
      continue
    else:
      f_name.append(f)
  f_name.sort()
  random.seed(load_all_seed)
  random.shuffle(f_name)
  
  count = 0
  for name in f_name:
    if count < p_test_num:
      img_test.append(cv2.resize(cv2.imread(path + name), (224,224), cv2.INTER_CUBIC))
      count += 1
    elif count < p_train_num + p_test_num:
      img_train.append(cv2.resize(cv2.imread(path + name), (224,224), cv2.INTER_CUBIC))
      count += 1
    else:
      break
  return img_train, img_test

#讀入增量資料
def open_image_from_Liteon_Charlie_gen(AorB, classNum, folder_ForG, sheet_num=0, load_all=False):
  img_x_train = []
  f_name = []
  path = ""
  if AorB != "AtransB":
    path = "/content/drive/MyDrive/Liteon_Charlie_gen/"+str(AorB)+"/"+ str(classNum) + "_" + str(folder_ForG) + "/"
  else:
    path = "/content/drive/MyDrive/Liteon_Charlie_gen/AtransB/"+str(classNum)+"/"

  for f in listdir(path):
    f_name.append(f)
  f_name.sort()
  random.seed(load_from_gen_seed)
  random.shuffle(f_name)
  
  if load_all == True:
    for name in f_name:
      img_x_train.append(cv2.resize(cv2.imread(path + name), (224,224), cv2.INTER_CUBIC))
  else:
    if sheet_num <= len(f_name):
      for i in range(sheet_num):
        img_x_train.append(cv2.resize(cv2.imread(path + f_name[i]), (224,224), cv2.INTER_CUBIC))
    else:
      print("要求超過數量! sheet_num需小於", str(len(f_name)))
      return None
  return img_x_train


In [ ]:
device = torch.device('cuda')
  
A_p_train_num = int(420) 
A_p_test_num = int(25)
  
B_p_train_num = int(85) 
B_p_test_num = int(5)

load_all_seed = int(6)
load_from_gen_seed = int(8)

from torch.utils.data import Dataset
class my_Dataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return self.images.shape[0]

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [ ]:
# 載入 A 資料集

img10_x_train, img10_x_test = load_all_img('A', 0, A_p_train_num, A_p_test_num)
img11_x_train, img11_x_test = load_all_img('A', 1, A_p_train_num, A_p_test_num)
img13_x_train, img13_x_test = load_all_img('A', 3, A_p_train_num, A_p_test_num)
img14_x_train, img14_x_test = load_all_img('A', 4, A_p_train_num, A_p_test_num)
  
img12_x_train, img12_x_test = load_all_img('A', 2, A_p_train_num, A_p_test_num)
img15_x_train, img15_x_test = load_all_img('A', 5, A_p_train_num, A_p_test_num)
img16_x_train, img16_x_test = load_all_img('A', 6, A_p_train_num, A_p_test_num)
img17_x_train, img17_x_test = load_all_img('A', 7, A_p_train_num, A_p_test_num)

# A 的2 5 6 7 使用翻轉圖片補資料

img12_x_train += open_image_from_Liteon_Charlie_gen('A', 2, "flip", 0, True)
img15_x_train += open_image_from_Liteon_Charlie_gen('A', 5, "flip", 0, True)
img16_x_train += open_image_from_Liteon_Charlie_gen('A', 6, "flip", 0, True)
img17_x_train += open_image_from_Liteon_Charlie_gen('A', 7, "flip", A_p_train_num-len(img17_x_train))

# A 的 2 5 6 使用gen圖片補齊資料

img12_x_train += open_image_from_Liteon_Charlie_gen('A', 2, "gen", A_p_train_num-len(img12_x_train))
img15_x_train += open_image_from_Liteon_Charlie_gen('A', 5, "gen", A_p_train_num-len(img15_x_train))
img16_x_train += open_image_from_Liteon_Charlie_gen('A', 6, "gen", A_p_train_num-len(img16_x_train))

p_train_num = A_p_train_num
p_test_num = A_p_test_num

img10_y_train = [0]*p_train_num
img10_y_test = [0]*p_test_num

img11_y_train = [1]*p_train_num
img11_y_test = [1]*p_test_num

img12_y_train = [2]*p_train_num
img12_y_test = [2]*p_test_num

img13_y_train = [3]*p_train_num
img13_y_test = [3]*p_test_num

img14_y_train = [4]*p_train_num
img14_y_test = [4]*p_test_num

img15_y_train = [5]*p_train_num
img15_y_test = [5]*p_test_num

img16_y_train = [6]*p_train_num
img16_y_test = [6]*p_test_num

img17_y_train = [7]*p_train_num
img17_y_test = [7]*p_test_num

#串接所有類別的訓練資料 (x,y)
img_x_train = img10_x_train + img11_x_train + img12_x_train + img13_x_train + img14_x_train + img15_x_train + img16_x_train + img17_x_train
img_y_train = img10_y_train + img11_y_train + img12_y_train + img13_y_train + img14_y_train + img15_y_train + img16_y_train + img17_y_train
  
print('len(img_x_train):',len(img_x_train))
print('len(img_y_train):',len(img_y_train))


#串接所有類別的測試資料 (x,y)
img_x_test = img10_x_test + img11_x_test + img12_x_test + img13_x_test + img14_x_test + img15_x_test + img16_x_test + img17_x_test
img_y_test = img10_y_test + img11_y_test + img12_y_test + img13_y_test + img14_y_test + img15_y_test + img16_y_test + img17_y_test
print('len(img_x_test):',len(img_x_test))
print('len(img_y_test):',len(img_y_test))


img_x_train = np.array(img_x_train)
img_x_test = np.array(img_x_test)

img_x_train = img_x_train.astype("float32")
for i in range(len(img_x_train)):
  img_x_train[i] = img_x_train[i]/255.0

img_x_test = img_x_test.astype("float32")
for i in range(len(img_x_test)):
  img_x_test[i] = img_x_test[i]/255.0

x_train_A = torch.zeros(len(img_x_train),3,224,224,dtype = torch.float32)
for i in range(len(img_x_train)): 
  x_train_A[i] = torch.from_numpy(img_x_train[i]).permute(2,1,0)#for BGR、HSV(cv2預設)

x_test_A = torch.zeros(len(img_x_test),3,224,224,dtype = torch.float32)
for i in range(len(img_x_test)): 
  x_test_A[i] = torch.from_numpy(img_x_test[i]).permute(2,1,0)#for BGR、HSV(cv2預設)

y_train_A = torch.from_numpy(np.array(img_y_train))
y_test_A = torch.from_numpy(np.array(img_y_test))

test_dataset_A = my_Dataset(x_test_A, y_test_A)

# 回傳x_validation

len(img_x_train): 3360
len(img_y_train): 3360
len(img_x_test): 200
len(img_y_test): 200


In [ ]:
# 讀入B所有資料
img10_x_train_B, img10_x_test_B = load_all_img('B', 0, B_p_train_num, B_p_test_num)
img11_x_train_B, img11_x_test_B = load_all_img('B', 1, B_p_train_num, B_p_test_num)
img12_x_train_B, img12_x_test_B = load_all_img('B', 2, B_p_train_num, B_p_test_num)
img13_x_train_B, img13_x_test_B = load_all_img('B', 3, B_p_train_num, B_p_test_num)
img14_x_train_B, img14_x_test_B = load_all_img('B', 4, B_p_train_num, B_p_test_num)
img15_x_train_B, img15_x_test_B = load_all_img('B', 5, B_p_train_num, B_p_test_num)
img16_x_train_B, img16_x_test_B = load_all_img('B', 6, B_p_train_num, B_p_test_num)
img17_x_train_B, img17_x_test_B = load_all_img('B', 7, B_p_train_num, B_p_test_num)


# B 的 1~7類從翻轉圖片補
img11_x_train_B += open_image_from_Liteon_Charlie_gen('B', 1, "filp", 0, True) # 只要所需訓練集大小不低於20張，此處皆不用改(flip一類有15張)
img12_x_train_B += open_image_from_Liteon_Charlie_gen('B', 2, "filp", 0, True)
img13_x_train_B += open_image_from_Liteon_Charlie_gen('B', 3, "filp", 0, True)
img14_x_train_B += open_image_from_Liteon_Charlie_gen('B', 4, "filp", 0, True)
img15_x_train_B += open_image_from_Liteon_Charlie_gen('B', 5, "filp", 0, True)
img16_x_train_B += open_image_from_Liteon_Charlie_gen('B', 6, "filp", 0, True)
img17_x_train_B += open_image_from_Liteon_Charlie_gen('B', 7, "filp", 0, True)

# B 的 1~7 使用gen圖片補齊資料

img11_x_train_B += open_image_from_Liteon_Charlie_gen('B', 1, "gen", B_p_train_num-len(img11_x_train_B))
img12_x_train_B += open_image_from_Liteon_Charlie_gen('B', 2, "gen", B_p_train_num-len(img12_x_train_B))
img13_x_train_B += open_image_from_Liteon_Charlie_gen('B', 3, "gen", B_p_train_num-len(img13_x_train_B))
img14_x_train_B += open_image_from_Liteon_Charlie_gen('B', 4, "gen", B_p_train_num-len(img14_x_train_B))
img15_x_train_B += open_image_from_Liteon_Charlie_gen('B', 5, "gen", B_p_train_num-len(img15_x_train_B))
img16_x_train_B += open_image_from_Liteon_Charlie_gen('B', 6, "gen", B_p_train_num-len(img16_x_train_B))
img17_x_train_B += open_image_from_Liteon_Charlie_gen('B', 7, "gen", B_p_train_num-len(img17_x_train_B))

p_train_num = B_p_train_num
p_test_num = B_p_test_num

img10_y_train_B = [0]*p_train_num
img10_y_test_B = [0]*p_test_num

img11_y_train_B = [1]*p_train_num
img11_y_test_B = [1]*p_test_num

img12_y_train_B = [2]*p_train_num
img12_y_test_B = [2]*p_test_num

img13_y_train_B = [3]*p_train_num
img13_y_test_B = [3]*p_test_num

img14_y_train_B = [4]*p_train_num
img14_y_test_B = [4]*p_test_num

img15_y_train_B = [5]*p_train_num
img15_y_test_B = [5]*p_test_num

img16_y_train_B = [6]*p_train_num
img16_y_test_B = [6]*p_test_num

img17_y_train_B = [7]*p_train_num
img17_y_test_B = [7]*p_test_num

#串接所有類別的訓練資料 (x,y)
img_x_train_B = img10_x_train_B + img11_x_train_B + img12_x_train_B + img13_x_train_B + img14_x_train_B + img15_x_train_B + img16_x_train_B + img17_x_train_B
img_y_train_B = img10_y_train_B + img11_y_train_B + img12_y_train_B + img13_y_train_B + img14_y_train_B + img15_y_train_B + img16_y_train_B + img17_y_train_B

print('len(img_x_train):',len(img_x_train_B))
print('len(img_y_train):',len(img_y_train_B))


# 打混訓練資料
index = [i for i in range(len(img_x_train_B))] 
random.shuffle(index)
index = np.array(index)
img_x_train_B = np.array(img_x_train_B)[index]
img_y_train_B = np.array(img_y_train_B)[index]

# 串接所有類別的測試資料 (x,y)
img_x_test_B = img10_x_test_B + img11_x_test_B + img12_x_test_B + img13_x_test_B + img14_x_test_B + img15_x_test_B + img16_x_test_B + img17_x_test_B
img_y_test_B = img10_y_test_B + img11_y_test_B + img12_y_test_B + img13_y_test_B + img14_y_test_B + img15_y_test_B + img16_y_test_B + img17_y_test_B
print('len(img_x_test_B):',len(img_x_test_B))
print('len(img_y_test_B):',len(img_y_test_B))

img_x_train_B = img_x_train_B.astype("float32")
for i in range(len(img_x_train_B)):
  img_x_train_B[i] = img_x_train_B[i]/255.0


for i in range(len(img_x_test_B)):
  img_x_test_B[i] = img_x_test_B[i]/255.0

def del_tensor_ele(arr,index1,index2, train_len):
  arr1 = arr[0:index1]
  arr2 = arr[index2:train_len]
  return torch.cat((arr1,arr2),dim = 0)

x_train_B = torch.zeros(len(img_x_train_B),3,224,224,dtype = torch.float32)
x_test_B = torch.zeros(len(img_x_test_B),3,224,224,dtype = torch.float32)
for i in range(len(img_x_train_B)): 
  x_train_B[i] = torch.from_numpy((img_x_train_B[i])).permute(2,1,0)#for BGR、HSV(cv2預設)
for i in range(len(img_x_test_B)): 
  x_test_B[i] = torch.from_numpy((img_x_test_B[i])).permute(2,1,0)#for BGR、HSV(cv2預設)


y_train_B = torch.from_numpy(np.array(img_y_train_B))
y_test_B = torch.from_numpy(np.array(img_y_test_B))

#交叉驗證處理
x_cross_train_B = []
x_cross_val_B = []
y_cross_train_B = []
y_cross_val_B = []

#10折交叉驗證
len_one_tenth_x_train = int(len(x_train_B)/10)

for i in range(10):
  x_buffer_train = x_train_B
  y_buffer_train = y_train_B
  x_cross_val_B.append(x_buffer_train[0+len_one_tenth_x_train*i:len_one_tenth_x_train+len_one_tenth_x_train*i])
  y_cross_val_B.append(y_buffer_train[0+len_one_tenth_x_train*i:len_one_tenth_x_train+len_one_tenth_x_train*i])
  x_cross_train_B.append(del_tensor_ele(x_buffer_train,0+len_one_tenth_x_train*i,len_one_tenth_x_train+len_one_tenth_x_train*i, len_one_tenth_x_train*10))
  y_cross_train_B.append(del_tensor_ele(y_buffer_train,0+len_one_tenth_x_train*i,len_one_tenth_x_train+len_one_tenth_x_train*i, len_one_tenth_x_train*10))


train_dataset_B = []
val_dataset_B = []
for i in range(10):
  train_dataset_B.append(my_Dataset(x_cross_train_B[i],y_cross_train_B[i]))
  val_dataset_B.append(my_Dataset(x_cross_val_B[i],y_cross_val_B[i]))

test_dataset_B = my_Dataset(x_test_B, y_test_B)


len(img_x_train): 680
len(img_y_train): 680
len(img_x_test_B): 40
len(img_y_test_B): 40


In [ ]:
dict_name_modules_A = {}
dict_name_modules_B = {}
load_all_seed = int(6)
load_from_gen_seed = int(8)

In [ ]:
#change
feature_A = []
feature_B = []
feature_tmp = []

In [ ]:
def dict_modules(model, AorB):
  count = 0
  global dict_name_modules_A, dict_name_modules_B
  for name,module in model.named_modules():
    #if 'conv' in name:
    #  dict_name_modules[name] = module
    #  count += 1
    if 'layer1.0.conv1' == name or 'layer2.0.conv1' == name or "layer3.0.conv1" == name:
      if AorB == 'A':
        dict_name_modules_A[name] = module
      elif AorB == 'B':
        dict_name_modules_B[name] = module
      count += 1
    

In [ ]:
#萃取feature_map
def feature_map(model, input, choose_layer, module_name, single, AorB):
  features = []

  def hook(module, input, output):
    features.append(input[0].clone().detach())
    del output

  global dict_name_modules_A, dict_name_modules_B, feature_tmp, feature_A, feature_B
  

  handle = []
  if AorB == 'A':
    handle.append(dict_name_modules_A[module_name].register_forward_hook(hook))
  elif AorB == 'B':
    handle.append(dict_name_modules_B[module_name].register_forward_hook(hook))

  model = model.to(device=torch.device('cuda')) # add
  input = input.to(device=torch.device('cuda'))
  y = model(input)

  for i in range(len(handle)):
    handle[i].remove()

  feature_tmp = features[0].cpu().detach()
  del features
  del handle

  torch.cuda.empty_cache()


  return feature_tmp

In [ ]:
random.seed(time.time())

In [ ]:
#change
def list_clear():
  global feature_tmp
  feature_tmp.clear()

In [ ]:
#選擇哪一層之後需要凍結
def freeze(model, choose_layer):
  count = 0
  for child in model.children():
    count+=1 
    if count > (choose_layer + 3): 
      for param in child.parameters():
        param.requires_grad = False


In [ ]:
#選擇哪一層之後需要解凍結
def unfreeze(model, choose_layer):
  count = 0
  for child in model.children():
    count+=1 
    if count > (choose_layer + 3): 
      for param in child.parameters():
        param.requires_grad = True

In [ ]:
class Discriminator(torch.nn.Module):
    def __init__(self, channels):
        super().__init__()
        # Filters [256, 512, 1024]
        # Input_dim = channels (Cx64x64)
        # Output_dim = 1
        self.main_module = nn.Sequential(
            # Omitting batch normalization in critic because our new penalized training objective (WGAN with gradient penalty) is no longer valid
            # in this setting, since we penalize the norm of the critic's gradient with respect to each input independently and not the enitre batch.
            # There is not good & fast implementation of layer normalization --> using per instance normalization nn.InstanceNorm2d()
            # Image (Cx32x32)


            nn.Conv2d(in_channels=channels, out_channels=256, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(256, affine=True),
            nn.LeakyReLU(0.2, inplace=True),

            # State (256x16x16)
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(512, affine=True),
            nn.LeakyReLU(0.2, inplace=True),


            # State (512x8x8)
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(1024, affine=True),
            nn.LeakyReLU(0.2, inplace=True))
            # output of main module --> State (1024x4x4)

        self.output = nn.Sequential(
            # The output of D is no longer a probability, we do not apply sigmoid at the output of D.
            nn.Conv2d(in_channels=1024, out_channels=1, kernel_size=4, stride=1, padding=0))


    def forward(self, x):
        x = self.main_module(x)

        return self.output(x)
        
    def feature_extraction(self, x):
        # Use discriminator for feature extraction then flatten to vector of 16384
        x = self.main_module(x)
        return x.view(-1, 1024*4*4)

In [ ]:
def gradient_penalty(D, xr, xf):
    """
    :param D:
    :param xr: [b, 2]
    :param xf: [b, 2]
    :return:
    """
    '''
    # [b, 1]
    t = torch.rand(batch_size, 1, 1, 1).cuda()
    # [b, 1] => [b, 2]  broadcasting so t is the same for x1 and x2
    t = t.expand_as(xr)
    # interpolation
    mid = t * xr + (1 - t) * xf
    # set it to require grad info
    mid.requires_grad_()
    pred = D(mid)
    grads = autograd.grad(outputs=pred, inputs=mid,
                          grad_outputs=torch.ones_like(pred),
                          create_graph=True, retain_graph=True, only_inputs=True)[0]
    gp = torch.pow(grads.norm(2, dim=1) - 1, 2).mean() * 10
    '''
    eta = torch.FloatTensor(batch_size,1,1,1).uniform_(0,1)
    eta = eta.expand(batch_size, xr.size(1), xr.size(2), xr.size(3))
    eta = eta.cuda()

    interpolated = eta * xr + ((1 - eta) * xf)
    interpolated = interpolated.cuda()
    interpolated = interpolated

    # define it to calculate gradient
    interpolated = Variable(interpolated, requires_grad=True)

    # calculate probability of interpolated examples
    prob_interpolated = D(interpolated)

    # calculate gradients of probabilities with respect to examples
    gradients = autograd.grad(outputs=prob_interpolated, inputs=interpolated,
                            grad_outputs=torch.ones(
                                prob_interpolated.size()).cuda() if torch.cuda.is_available() else torch.ones(
                                prob_interpolated.size()),
                            create_graph=True, retain_graph=True)[0]

    grad_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * 10
    return grad_penalty

    #return gp

In [ ]:
h_dim = 400
batch_size = 32

In [ ]:
#依batch_size讀入a資料(有iterator特性)
def data_generator_A(x_train_A):
  global e, batch_size
  train_loader = torch.utils.data.DataLoader(my_Dataset(x_train_A, y_train_A), batch_size = batch_size, shuffle= True,num_workers=4, pin_memory=True, drop_last=True)
  while True:
    for imgs,labels in train_loader:
      yield imgs, labels

In [ ]:
#依batch_size讀入b資料(有iterator特性)
def data_generator_B(x_train_B):
  global batch_size
  train_loader = torch.utils.data.DataLoader(my_Dataset(x_train_B, y_train_B), batch_size = batch_size, shuffle= True,num_workers=4, pin_memory=True, drop_last=True)
  while True:
    for imgs,labels in train_loader:
      yield imgs, labels

In [ ]:
#權重初始化
def weights_init(m):
    classname = m.__class__.__name__
    print('classname:', classname)

    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('InstanceNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
#計算正確預測量
def compute_correct(model, train_loader, loss_fn):
  correct = 0
  total = 0
  with torch.no_grad():
    for imgs, labels in train_loader:
        imgs = imgs.to(device = device)
        labels = labels.to(device = device)
        outputs = model(imgs)
        predicted = torch.max(outputs,dim = 1)
        total += labels.shape[0]
        correct += int((predicted.indices == labels).sum())
  
  return correct/total

def training_loop(n_epochs,optimizer,model,loss_fn,train_dataset):
  train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 32,shuffle= True,num_workers=4, pin_memory=True, drop_last=True)
  
  for epoch in range(0,n_epochs):
    #批次
    
    loss_train = 0.0
    count = 1
    model.train()  #change
    for imgs,labels in train_loader:
      imgs = imgs.to(device = device)
      labels = labels.to(device = device)
      #print(count)
      outputs = model(imgs)

      r_loss = loss_fn(outputs,labels)
      regularization_loss = 0
      for param in model.parameters():
        regularization_loss += torch.sum(param.pow(2.0))

      loss = r_loss + 0.0001 * regularization_loss

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += r_loss.item()

    model.eval()  #change
    correct = compute_correct(model, train_loader, loss_fn)#, val_correct, val_loss
    
    print('Epoch {}, Training loss {}, Training accuracy {}'.format(epoch + 1, loss_train/len(train_loader), correct))

In [ ]:
model_name = '/content/drive/MyDrive/charliesModel/420_35ep_satu.pt'   #選擇source model

choose_layer = 1    #選擇要萃取哪一卷積層的特徵圖   
                    #0 -> 'layer1.0.conv1, 1 -> layer2.0.conv1, 2 -> 'layer3.0.conv1
                    #0 -> 64 1 -> 256 2 -> 512 (每一層channel 數)

module_name = None
d_input_ch = None

if choose_layer == 0:
  module_name = 'layer1.0.conv1'
  d_input_ch = 64  #d_input_ch指Discriminator的input channel數
elif choose_layer == 1:
  module_name = 'layer2.0.conv1'
  d_input_ch = 256
elif choose_layer == 2:
  module_name = 'layer3.0.conv1'
  d_input_ch = 512


In [ ]:
torch.manual_seed(23)
np.random.seed(23)
device = torch.device('cuda')
all_imags = None

model_A = models.resnet50(pretrained=False)  #change
fc_feature = model_A.fc.in_features    #change
model_A.fc = torch.nn.Linear(fc_feature,8)  #change
model_A.load_state_dict(torch.load(model_name)) 
#model_A = torch.load(model_name)
model_A = model_A.to(device=torch.device("cuda"))

dict_modules(model_A, 'A')

model_B = copy.deepcopy(model_A)
model_B = model_B.to(device=torch.device("cuda"))

In [ ]:
#用來儲存模型
torch.save(model_B,'/content/drive/My Drive/420_20ep_light.pt')

In [ ]:
#再開始訓練之前，先凍結，因為先更新generator，才使用分類loss更新整個B模型
freeze(model_B, choose_layer)

In [ ]:
from torch.nn.common_types import T
from torch.optim.lr_scheduler import LambdaLR
D = Discriminator(d_input_ch).to(device)
D.apply(weights_init)
e = 0
data_iter_A = data_generator_A(x_train_A)  #讀入第一批A資料
data_iter_B = data_generator_B(x_train_B)  #讀入第一批B資料

optim_G = torch.optim.Adam(model_B.parameters(), lr=1e-6, betas=(0.5, 0.9))
optim_D = torch.optim.Adam(D.parameters(), lr=1e-6, betas=(0.5, 0.9))
optimizer = torch.optim.Adam(model_B.parameters(),lr = 1e-6, betas=(0.5, 0.9))

scheduler_D = torch.optim.lr_scheduler.ExponentialLR(optim_D, gamma=0.8) 

one = torch.tensor(1, dtype=torch.float)
mone = one * -1
one = one.cuda()
mone = mone.cuda()

loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(5):
    e = epoch
    dict_modules(model_B, 'B')
    # Requires grad, Generator requires_grad = False
    for p in D.parameters():
        p.requires_grad = True
    d_loss_real = 0
    d_loss_fake = 0
    Wasserstein_D = 0

    # 1. train D first
    for _ in range(3):  # train D 5 times, adjustable
      for p in D.parameters():
        p.requires_grad = True

      D.zero_grad()

      # 1.1 train on real data
      a, a_labels = next(data_iter_A)
      a = a.to(device)
      a_labels = a_labels.to(device)

      xr = feature_map(model_B, a, choose_layer, module_name, 'B').cuda() #萃取A資料特徵圖
      # Train discriminator
      # WGAN - Training discriminator more iterations than generator
      # Train with real images
      d_loss_real = D(xr)  #餵給Discriminator
      d_loss_real = d_loss_real.mean() #計算期望值
      d_loss_real.backward(mone)

      b, b_labels = next(data_iter_B)
      b = b.to(device)
      b_labels = b_labels.to(device)
      xf = feature_map(model_B, b, choose_layer, module_name, 'B').cuda() #萃取B資料特徵圖

      d_loss_fake = D(xf)  #餵給Discriminator
      d_loss_fake = d_loss_fake.mean() #計算期望值
      d_loss_fake.backward(one)

      # Train with gradient penalty
      gp = gradient_penalty(D, xr, xf.detach())
      gp.backward()
      
      d_loss = d_loss_fake - d_loss_real + gp
      Wasserstein_D = d_loss_real - d_loss_fake
      optim_D.step()  #更新Discriminator

      del xr


      # Generator update
      for p in D.parameters():
        p.requires_grad = False  # to avoid computation

      correct = 0
      total = 0

      model_B.zero_grad()
      D.zero_grad()
      
      # train generator
      # compute loss with fake images
      g_loss = D(xf)
      g_loss = g_loss.mean()
      g_loss = g_loss.requires_grad_()
      g_loss.backward(mone)    #更新Generator(模型前部分)
      
      
      unfreeze(model_B, choose_layer)  #解凍，以便使用分類loss更新整個模型

      c = torch.cat((a,b), 0)
      c_labels = torch.cat((a_labels, b_labels))
      
      pred_labels = model_B(c)
      classifer_loss = loss_fn(pred_labels,b_labels)
      regularization_loss = 0
      for param in model_B.parameters():
        regularization_loss += torch.sum(param.pow(2.0))

      loss = classifer_loss #+ 0.0001 * regularization_loss
      loss.backward()

      optimizer.step()   #更新整體模型
      optimizer.zero_grad()   
      
      freeze(model_B, choose_layer)  #凍結模型後部分

      model_B.eval()
      predicted = torch.max(pred_labels,dim = 1)
      total += c_labels.shape[0]
      correct += int((predicted.indices == c_labels).sum())
      model_B.train()
      del c
      del c_labels
      torch.cuda.empty_cache()

      

    if epoch % 5 == 0:
      scheduler_D.step()
    
    if epoch % 1 == 0:
        print("lossr:", d_loss_real.item(), " lossf:", d_loss_fake.item(), " gp:", gp.item())#,"sigma:", sigma_loss.item()
        print("epoch:", epoch, " loss_D:", d_loss.item()," loss_G:", g_loss.item(), " loss_class:", classifer_loss.item(), "acc:", correct / total,"\n")

In [ ]:
#計算準確率

test_total = 0
test_loss_train = 0.
total_accuracy = 0.
test_correct = 0

test_dataset = my_Dataset(x_test_B, y_test_B)
test_loader = torch.utils.data.DataLoader(test_dataset , batch_size = 1,shuffle = False,num_workers=4, pin_memory=True, drop_last=True)

loss_fn = torch.nn.CrossEntropyLoss()
error_label_counetr = [0]*8
model_B.eval()
with torch.no_grad():
  for imgs, labels in test_loader:  #一個for跑20張圖
    imgs = imgs.to(device = device)
    labels = labels.to(device = device)
    test_outputs = model_B(imgs)  #CHANGE
    test_loss = loss_fn(test_outputs,labels)

    test_loss_train += test_loss.item()
    test_predicted = torch.max(test_outputs,dim = 1) #https://blog.csdn.net/cunchi4221/article/details/107471005
    test_total += labels.shape[0]
    test_correct += int((test_predicted.indices == labels).sum())
    
  print(test_correct, test_total)
  total_accuracy += test_correct/test_total

  print(total_accuracy)